In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("ProjetoFinal") \
    .config("spark.jars.packages", 
            "org.mongodb.spark:mongo-spark-connector_2.12:10.4.0,"
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/myuser/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/myuser/.ivy2/cache
The jars for the packages stored in: /home/myuser/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8c96903e-4525-4a28-87c0-97f0880fcb9b;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.4.0 in central
#mongodb-driver-sync;5.1.4 in central4.0 in central

	[5.1.4] org.mongodb#mongodb-driver-sync;[5.1.1,5.1.99)
	found org.mongodb#bson;5.1.4 in central in central

	[5.1.4] org.mongodb#mongodb-driver-sync;[5.1.1,5.1.99)

mongodb-driver-core;5.1.4 in central5.1.4] org.mongodb#mongodb-driver-sync;[5.1.1,5.1.99)


	found org.mongodb#bson-record-codec;5.1.4 in centralb#mongodb-driver-sync;[5.1.1,5.1.99)



apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in centralver-sync;[5.1.1,5.1.99)




	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in cent

In [2]:
df = spark.read.csv("file:///home/myuser/myfiles/discentes-2023.csv", sep=";", header=True, inferSchema=True)
print(f"\n \t Leu o CSV")

df.write.format("mongodb") \
    .option("connection.uri", "mongodb://mongodb:27017/engdados") \
    .option("database", "engdados") \
    .option("collection", "ingressantes") \
    .mode("overwrite") \
    .save()
print(f"\n \t Dados dos ingressantes 2023 salvos na coleção ingressantes")

                                                                                                                                                       [Stage 1:>                                                          (0 + 2) / 2]                                                                                =============================>                             (1 + 1) / 2]                                                                                [Stage 1:>                                                          (0 + 2) / 2]                                                                                


 	 Leu o CSV


[Stage 2:=============================>                             (1 + 1) / 2]


 	 Dados dos ingressantes 2023 salvos na coleção ingressantes


In [3]:
df_kafka_raw = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "mongo.engdados.ingressantes") \
    .option("startingOffsets", "earliest") \
    .load()

In [4]:
df_kafka_string = df_kafka_raw.selectExpr("CAST(value AS STRING) as json_str")
print(f"\n \t CAST realizado")


 	 CAST realizado


In [5]:
df_after = df_kafka_string.select(get_json_object(col("json_str"), "$.payload.after").alias("after_json"))
print(f"\n \t Dados do after separados")


 	 Dados do after separados


In [6]:
stream = df_after.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .start()

25/07/19 23:07:03 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0605023d-1a9a-406d-8b72-2dfd58c15ccb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/07/19 23:07:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/07/19 23:07:04 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/07/19 23:07:04 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/07/19 23:07:04 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/07/19 23:07:04 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known con

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|after_json                                                                                                                                                                                                              

In [8]:
schema_flat = StructType([
    StructField("matricula", StringType()),
    StructField("ano_ingresso", StringType()),
    StructField("periodo_ingresso", StringType()),
    StructField("id_curso", StringType()),
    StructField("id_unidade", StringType()),
    StructField("id_unidade_gestora", StringType()),
    StructField("nome_discente", StringType()),
    StructField("sexo", StringType()),
    StructField("forma_ingresso", StringType()),
    StructField("tipo_discente", StringType()),
    StructField("status", StringType()),
    StructField("sigla_nivel_ensino", StringType()),
    StructField("nivel_ensino", StringType()),
    StructField("nome_curso", StringType()),
    StructField("modalidade_educacao", StringType()),
    StructField("nome_unidade", StringType()),
    StructField("nome_unidade_gestora", StringType())
])

df_2023 = df_after.select(from_json(col("after_json"), schema_flat).alias("data")).select("data.*")
print(f"\n \t Dados vindos do stream estruturados")


 	 Dados vindos do stream estruturados


In [9]:
df_2024 = spark.read.csv("file:///home/myuser/myfiles/discentes-2024.csv", sep=";", header=True, inferSchema=True)

print(f"\n \t Dados dos ingressantes 2024 carregados no dataframe")

[Stage 6:>                                                          (0 + 1) / 1]                                                                                


 	 Dados dos ingressantes 2024 carregados no dataframe


In [10]:
df_2023.writeStream \
    .foreachBatch(lambda batch_df, batch_id: batch_df.alias("a")
                  .join(df_2024.alias("b"), on=[col("a.nome_discente") == col("b.nome_discente"), col("a.nivel_ensino") == col("b.nivel_ensino")], how="inner")
                  .select(
                      col("a.nome_discente").alias("nome_discente"),
                      col("a.sexo").alias("sexo"),
                      struct(
                          col("a.matricula"),
                          col("a.id_curso"),
                          col("a.nome_curso"),
                          col("a.forma_ingresso"),
                          col("a.modalidade_educacao")
                      ).alias("ano_2023"),
                      struct(
                          col("b.matricula"),
                          col("b.id_curso"),
                          col("b.nome_curso"),
                          col("b.forma_ingresso"),
                          col("b.modalidade_educacao")
                      ).alias("ano_2024")
                  )
                  .write.format("mongodb") \
                  .option("connection.uri", "mongodb://mongodb:27017/engdados") \
                  .option("database", "engdados") \
                  .mode("overwrite")
                  .option("collection", "reingressantes")
                  .save()) \
    .outputMode("append") \
    .start()

print(f"\n \t Transformações realizadas, dados salvos na collecction reingressantes")


 	 Transformações realizadas, dados salvos na collecction reingressantes


25/07/19 23:10:17 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8e26e3eb-6122-43c1-a8f9-bfab96217470. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/07/19 23:10:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/07/19 23:10:17 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/07/19 23:10:17 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/07/19 23:10:17 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/07/19 23:10:17 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but                  